## 初期設定

In [15]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.data_cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


In [16]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw',
  'interim': './data/interim',
  'processed': './data/processed',
  'reference': './data/reference'},
 'files': {'raw': ['fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv',
   'fy-stock-dividend.csv'],
  'reference': 'CodeData.csv',
  'interim': None,
  'processed': None},
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'na_values': ['', '-'],
 'dtype_chk': {'raw': {'コード': 'string',
   '年度': 'datetime64[ns]',
   '総資産': 'Float64',
   '純資産': 'Float64',
   '株主資本': 'Float64',
   '利益剰余金': 'Float64',
   '短期借入金': 'Float64',
   '長期借入金': 'Float64',
   'BPS': 'Float64',
   '自己資本比率': 'Float64',
   '自己資本': 'Float64',
   '売上高': 'Float64',
   '営業利益': 'Float64',
   '経常利益': 'Float64',
   '純利益': 'Float64',
   'EPS': 'Float64',
   'ROE': 'Float64',
   'ROA': 'Float64',
   '営業CF': 'Float64',
   '投資CF': 'Float64

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認し、ファイルロードからクリーニングまでのプロセスを決定します。

<details>
<summary><b>結果</b></summary>


</details>


### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめ、プロセスを確定します。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。  
修正処理は不要です。
</details>

In [17]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\n次に欠けてるファイルがないか確認します")
if all(dl.chk_file_missing(df) for df in df_DATAs_BY_ALL_FILEs.values()):
    print("✅ ファイル欠損はありませんでした。")

print("\n重複ファイルがないか確認します")
duplicates = dl.chk_duplicate_dfs(df_DATAs_BY_ALL_FILEs, False,False)
if duplicates:
    print("🔍 重複データフレームが見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for key in key_group:
            print(key)
else:
    print("✅ 重複データフレームはありません。")
print("\nデータロードの確認を終了します。")



ファイル毎にデータをロードします

次に欠けてるファイルがないか確認します
✅ ファイル欠損はありませんでした。

重複ファイルがないか確認します
✅ 重複データフレームはありません。

データロードの確認を終了します。


### 欠損値表現の確認と処理
欠損値の表現を確認し、修正プロセスを決定します。

<details>
<summary><b>結果</b></summary>

"-"が欠損値として使われています。  
これはsetting.yamlのnan_valuesに登録し欠損値として読み込みます。  
  
また、ゼロも使われています。  
ゼロは値として残し、必要に応じてfeatures_generatorで再計算し更新します。

以下がゼロが使われている項目です。  
{'fy-balance-sheet.csv': ['BPS', '短期借入金', '総資産', '自己資本比率', '長期借入金']  
'fy-cash-flow-statement.csv': ['営業CF', '営業CFマージン', '投資CF', '現金同等物', '設備投資', '財務CF']  
'fy-profit-and-loss.csv': ['EPS', 'ROA', 'ROE', '営業利益', '売上高', '純利益', '経常利益'],  
'fy-stock-dividend.csv': ['一株配当', '剰余金の配当', '純資産配当率', '総還元性向', '自社株買い', '配当性向']}
</details>

In [18]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename,header=1)
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}
#display(df_DATAs_BY_ALL_FILEs)

print("\nファイルごとに各文字列の個数をカウントします")
df_placeholder_counts = pd.DataFrame()
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = cu.chk_missing_values_expression(df, filename, year)
    df_placeholder_counts = pd.concat([df_placeholder_counts,df_tmp], axis=0)
#display(df_placeholder_counts)

print("\n欠損値と疑われる文字と該当する列を出力します。")
result = cu.chk_missing_and_suspect(df_placeholder_counts)
# 例: "欠損コードA" に該当する列リスト
#display(result)


ファイル毎にデータをロードします

ファイルごとに各文字列の個数をカウントします

欠損値と疑われる文字と該当する列を出力します。
-           167782
0             4155
alphabet      1940
dtype: object


In [19]:
# ゼロ値のチェック
#del list
df_merge = defaultdict(list)
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_merge[filename].append(df)
# ファイルごとに結合（行方向）
df_final = {fname: pd.concat(dfs, ignore_index=True) for fname, dfs in df_merge.items()}
#display(df_final["fy-balance-sheet.csv"])

print("\nファイルごとにゼロを含むコードをチェックします。")
filename = settings["files"]["raw"][0]
print(f"対象ファイル: {filename}")

df_all = df_final[filename]
result_list = []  # 結果を格納するリスト
for col in df_all:
    df_zero = df_all[df_all[col] == "0"]
    codes_with_zero = df_zero["コード"].unique()
    for code in codes_with_zero:
        df_code = df_all.query("コード == @code").copy()
        df_code["対象列"] = col
        result_list.append(df_code)
# すべての結果をまとめる
if result_list:
    df_result = pd.concat(result_list, ignore_index=True)
else:
    print("ゼロを含むコードは見つかりませんでした。")
display(df_result[df_result["対象列"] == "総資産"])


ファイルごとにゼロを含むコードをチェックします。
対象ファイル: fy-balance-sheet.csv


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率,対象列
0,9164,2020/12,0,-,-,-,-,-,-,-1481,総資産
1,9164,2021/12,68349000000,17372000000,17372000000,-533000000,-,32963000000,173.72,25.4,総資産
2,9164,2022/12,74638000000,21026000000,21026000000,3088000000,800000000,31620000000,210.26,28.2,総資産
3,9164,2023/12,79947000000,25927000000,25927000000,7989000000,-,29648000000,259.27,32.4,総資産
4,9164,2024/12,77563000000,27845000000,27845000000,10906000000,-,25765000000,278.45,35.9,総資産
5,9164,2024/12,77563000000,27845000000,27845000000,10906000000,-,25765000000,278.45,35.9,総資産


In [20]:
# アルファベット値のチェック
print("")
print("アルファベットが含まれているコードを出力します。")
code_list = []
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = df[df["コード"].astype(str).str.contains("[A-Za-z]", na=False)]
    code_list = code_list + df_tmp["コード"].unique().tolist()
print(code_list)


アルファベットが含まれているコードを出力します。
['297A', '297A', '297A', '297A', '297A', '297A', '297A', '281A', '297A', '130A', '138A', '141A', '142A', '145A', '146A', '148A', '149A', '153A', '155A', '184A', '190A', '281A', '297A', '130A', '137A', '138A', '141A', '142A', '143A', '145A', '146A', '147A', '148A', '149A', '150A', '151A', '153A', '155A', '156A', '157A', '160A', '168A', '173A', '175A', '176A', '177A', '184A', '186A', '189A', '190A', '192A', '194A', '195A', '196A', '197A', '206A', '211A', '212A', '215A', '218A', '219A', '220A', '228A', '241A', '242A', '244A', '245A', '246A', '248A', '259A', '260A', '261A', '262A', '264A', '265A', '269A', '276A', '280A', '281A', '285A', '286A', '290A', '297A', '298A', '319A', '130A', '137A', '138A', '141A', '142A', '143A', '145A', '146A', '147A', '148A', '149A', '150A', '151A', '153A', '155A', '156A', '157A', '160A', '168A', '173A', '175A', '176A', '177A', '184A', '186A', '189A', '190A', '192A', '194A', '195A', '196A', '197A', '206A', '211A', '212A', '215A', '218A

### データ型の整合性確認
ロードしたファイルの方が各列で共通になるようにします。
<details>
<summary><b>結果</b></summary>
以下のように処理しています。

- コード：string
- 年度：timestump
- その他：float
</details>

In [21]:
print("\nファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = {
    (filename, year): dl.load_on_startup(settings["data_path"]["raw"], str(year), filename, header=1, na_values=settings["na_values"])
    for filename in settings["files"]["raw"]
    for year in settings["years"]
}


ファイル毎にデータをロードします


In [22]:
print("\n読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ")
for keys,df in df_DATAs_BY_ALL_FILEs.items():
    object_cols = df.select_dtypes(include='object').columns
    df_DATAs_BY_ALL_FILEs[keys].loc[:, object_cols] = df.loc[:, object_cols].replace("-", np.nan)



読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ


In [23]:
print("\nコードと年度以外をfloat化")
except_codes= ["コード", "年度"]
df_type_change = {
    key: cu.convert_columns_type(df, df.drop(columns=except_codes, errors="ignore").columns, "float", False)
    for key, df in df_DATAs_BY_ALL_FILEs.items()
}


コードと年度以外をfloat化


In [24]:
print("\nコードをstring化")
df_type_change = {
    key: df.assign(コード=df["コード"].astype("string"))
    for key, df in df_type_change.items()
}


コードをstring化


In [25]:
print("\n年度をdatetime化")
df_type_change = {
    key: df.assign(年度=pd.to_datetime(df["年度"], format="%Y/%m"))
    for key, df in df_type_change.items()
}


年度をdatetime化


In [26]:
for (filename, year), df in df_type_change.items():
    print(filename, year)
    cu.chk_finale_dtype(df, settings["dtype_chk"]["raw"])

fy-balance-sheet.csv 2010
fy-balance-sheet.csv 2011
fy-balance-sheet.csv 2012
fy-balance-sheet.csv 2013
fy-balance-sheet.csv 2014
fy-balance-sheet.csv 2015
fy-balance-sheet.csv 2016
fy-balance-sheet.csv 2017
fy-balance-sheet.csv 2018
fy-balance-sheet.csv 2019
fy-balance-sheet.csv 2020
fy-balance-sheet.csv 2021
fy-balance-sheet.csv 2022
fy-balance-sheet.csv 2023
fy-balance-sheet.csv 2024
fy-balance-sheet.csv 2025
fy-cash-flow-statement.csv 2010
fy-cash-flow-statement.csv 2011
fy-cash-flow-statement.csv 2012
fy-cash-flow-statement.csv 2013
fy-cash-flow-statement.csv 2014
fy-cash-flow-statement.csv 2015
fy-cash-flow-statement.csv 2016
fy-cash-flow-statement.csv 2017
fy-cash-flow-statement.csv 2018
fy-cash-flow-statement.csv 2019
fy-cash-flow-statement.csv 2020
fy-cash-flow-statement.csv 2021
fy-cash-flow-statement.csv 2022
fy-cash-flow-statement.csv 2023
fy-cash-flow-statement.csv 2024
fy-cash-flow-statement.csv 2025
fy-profit-and-loss.csv 2010
fy-profit-and-loss.csv 2011
fy-profit-and-lo

### 初期品質の確認

指標となる以下の数を確認し、必要な処理のプロセスを決定します。

- コード
- 年度
- 列

<summary><b>結果</b></summary>

</details>

In [27]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)
print("\n処理済みデータをロードします。")
df_PROCESSED = df_type_change
#display(df_PROCESSED)


print("\n企業コード数、エンドの数、列数をファイルごとにまとめます。")
df_file_info = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED.items()
])
#display(df_file_info)

print("\n各ファイルの年推移を可視化します。")
for col in ["code_counts", "year_counts", "column_counts"]:
    fig = px.line(df_file_info, x="year", y=col, color="file", title=col)
    #fig.show()

print("\n最新年のファイルに登録されている年度を表示します。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("\n最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"]["コード"])

print("\nコードを限定し、過去のデータファイルと同じかどうか調べます。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_PROCESSED[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("\n最新年度にある過去のデータを取得し、過去のデータを更新します")
df_PROCESSED_AFTER = cu.update_duplicated(df_PROCESSED, 2025)

print("\n正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2025)]
display(df[df["コード"]=="130A"])
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2024)]
display(df[df["コード"]=="130A"])

print("\n最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
df_PROCESSED_AFTER = {
    (file, year): (df[df["年度"] >= cutoff_date] if year == 2025 else df)
    for (file, year), df in df_PROCESSED_AFTER.items()
}

print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。")
df_file_info_after = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED_AFTER.items()
])
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()

df_file_info = pd.DataFrame([
    {"file": filename, "year": year, "code_counts": df["コード"].nunique(), "year_counts": df["年度"].nunique(), "column_counts": df.columns.nunique()}
    for (filename, year), df in df_PROCESSED.items()
])
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).reset_index()
display(df_compare)
fig = px.line(df_compare, x="year", y="code_counts", color="file", line_dash="区分", markers=True)
fig.update_layout(title="処理前後のコード数推移", xaxis_title="年度", yaxis_title="コード数")
fig.show()


処理済みデータをロードします。

企業コード数、エンドの数、列数をファイルごとにまとめます。

各ファイルの年推移を可視化します。

最新年のファイルに登録されている年度を表示します。

最新年に登録されている過去の年度と過去のデータが同じか確認します
まずは、年度を限定して、コードの一覧を見ます

コードを限定し、過去のデータファイルと同じかどうか調べます。

最新年度にある過去のデータを取得し、過去のデータを更新します

正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
1,130A,2024-12-01,2248958000.0,2209548000.0,2209548000.0,-203393000.0,<NA>,<NA>,340.61,98.2


,コード,年度,総資産,純資産,株主資本,利益剰余金,短期借入金,長期借入金,BPS,自己資本比率
1,130A,2024-12-01,2248000000.0,2209000000.0,2209548000.0,-203393000.0,<NA>,<NA>,340.61,98.2



最後に最新年度にある過去のデータを消去します。

各ファイルの年推移をもう一度可視化し、改善していることを確かめます。


,file,year,code_counts,year_counts,column_counts,区分
0,fy-balance-sheet.csv,2010,3406,12,10,処理前
1,fy-balance-sheet.csv,2011,3513,12,10,処理前
2,fy-balance-sheet.csv,2012,3589,12,10,処理前
3,fy-balance-sheet.csv,2013,3679,12,10,処理前
4,fy-balance-sheet.csv,2014,3780,12,10,処理前
...,...,...,...,...,...,...
123,fy-stock-dividend.csv,2021,3618,12,8,処理後
124,fy-stock-dividend.csv,2022,3606,12,8,処理後
125,fy-stock-dividend.csv,2023,3606,12,8,処理後
126,fy-stock-dividend.csv,2024,3574,12,8,処理後


### 上場企業データのクリーニング

<summary><b>結果</b></summary>

</details>

In [28]:
print("\nロードします。")
df_code_info = dl.load_on_startup(settings["data_path"]["reference"], "", settings["files"]["reference"],header=0,na_values=settings["na_values"])

dl.chk_file_missing(df_code_info)

print("\n欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(
    df_code_info, filename=settings["files"]["reference"], option_value=""
)
#display(df_missing_values_expression)

print("\n読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ")
object_cols = df_code_info.select_dtypes(include='object').columns
df_code_info.loc[:, object_cols] = df_code_info.loc[:, object_cols].replace("-", np.nan)
#display(df_code_info)

print("\n数値列を Int64 に変換します。")
numeric_cols = ["17業種コード", "規模コード"]
df = df_code_info
df = cu.convert_columns_type(df, numeric_cols, "int", True)

print("\nstring 型に統一します。")
df = df.assign(**{col: df[col].astype("string") for col in df.columns if col not in numeric_cols})

print("\n日付列を datetime に変換します。")
df["日付"] = pd.to_datetime(df["日付"], errors="coerce", format="%Y%m%d")

df_type_change = df

print("\ndtypeの最終チェックをします。")
cu.chk_finale_dtype(df_type_change, settings["dtype_chk"]["reference"])

print("\n欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(df_type_change, filename=settings["files"]["reference"], option_value="")
#display(df_missing_values_expression)

print("\nデータ型を確認します。")
df_type = cu.chk_dtype(df=df_type_change,filename=settings["files"]["reference"],option_value="",na_drop=True)
#display(df_type)

#display(df_type_change)


ロードします。

欠損値表現を確認します。

読み込みで処理できなかった残りの文字列カラムに対し、replaceでクリーンアップ

数値列を Int64 に変換します。

列 '17業種コード' を int に変換中...

列 '規模コード' を int に変換中...

string 型に統一します。

日付列を datetime に変換します。

dtypeの最終チェックをします。

欠損値表現を確認します。

データ型を確認します。
